In [1]:
import pandas as pd
from time import time
import re
import string
import joblib

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [3]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from cleantext import clean
import pyLDAvis
import matplotlib.pyplot as plt
import pyLDAvis.sklearn

In [4]:
RANDOM_SEED=123
N_FEATURES = 1000

In [5]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'same', "hadn't", 'doesn', 'again', 'between', 'the', 'more', 'hers', 'to', 'my', 'both', 'has', "you're", 'or', 're', 'needn', 'over', 'that', 'have', 'because', 'won', 'here', 'd', 'doing', 's', 'don', 'i', 'theirs', 'he', 'be', 'nor', 'each', 'where', 'not', 'ourselves', 'what', "you'd", 'been', 'who', 'll', 'yourself', 'should', "don't", "should've", 'myself', "shan't", 'for', "couldn't", 'but', 'above', 'as', 'shan', 'when', 'by', 'after', 'were', "it's", 'his', 'isn', 'o', "doesn't", 'ma', 'me', 'why', 'of', 'they', 'it', "she's", 'about', "shouldn't", 'up', 'and', 'an', "wouldn't", 'she', 'did', 'weren', "won't", 'will', 'most', 'further', 'only', 'at', 'themselves', 'you', "that'll", 'into', 'too', "weren't", 'out', 'aren', 'with', 'if', 'these', 'herself', 'now', "you've", 'few', 'other', 'their', 'in', 'its', 'there', 'mustn', 'while', 'below', 'itself', 'such', 'yours', 'couldn', 'are', 'himself', 'being', 'all', 'am', 'having', 'which', 'during', 'y', 'own', 'we', 'is', 't

## Очистка текстов

In [6]:
def tolower(s):
    return s.lower()

def remove_code_blocks(s):
    return re.sub(r"```[^\S\r\n]*[a-z]*\n.*?\n```", '', s, 0, re.DOTALL)

def remove_long_words(s, good_len=15):
    return " ".join([word for word in s.split() if len(word) < good_len])

def remove_stopw(s):
    return " ".join([word for word in s.split() if word not in stop_words])

def remove_custom_stopw(s):
    stopw = ["line", "file", "jetbrains", "idea", "pycharm", "type", "import", "class"]
    stopw += ["type", "int", "float", "python", "run", "traceback", "version", ""]
    
    return " ".join([word for word in s.split() if word not in stopw])

def remove_unicode(s):
    text_encode = s.encode(encoding="ascii", errors="ignore")
    text_decode = text_encode.decode()
    clean_text = " ".join([word for word in text_decode.split()])
    return clean_text

def remove_stuff(s):
    s = re.sub(r"@\S+", "", s)
    s = re.sub(r"\$", "", s)
    s = re.sub(r"https?:\/\/.*[\r\n]*", "", s)
    s = re.sub(r"#", "", s)
    punct = set(string.punctuation) 
    s = "".join([ch for ch in s if ch not in punct])
    return s

def stem_lem(s):
    stemmer = PorterStemmer()
    s = " ".join([stemmer.stem(word) for word in s.split()])
    return s

def remove_java_packages(s):
     return " ".join([word for word in s.split() if not len(word.split('.')) > 2])
    
def clear_str(s):
    s = remove_code_blocks(s)
    s = tolower(s)
    s = remove_java_packages(s)
    s = remove_stopw(s)
    s = remove_custom_stopw(s)
    s = remove_unicode(s)
    s = remove_stuff(s)
    s = stem_lem(s)
    s = remove_long_words(s)
    return s

In [7]:
data = pd.read_json('pycharm_issues.json', lines=True)

In [8]:
def clean_data_descr(s):
    if not s:
        return ""
    s = clear_str(s)
    
    s = clean(f"{s}",
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=True,      # replace all currency symbols with a special token
    no_punct=True,                 # remove punctuations
    no_emoji=True,
    
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_digit="",
    replace_with_currency_symbol="",
    lang="en"                       # set to 'de' for German special handling
)
    return s

In [9]:
cleaning_lambda = lambda d: clean_data_descr(d)

In [10]:
t0 = time()

data["str_data"] = data["summary"].astype(str) + data["description"].astype(str)
data["str_data"] = data["str_data"].apply(cleaning_lambda)


time() - t0

25.19657301902771

### Получим данные про версии 2020.2 и 2020.3

In [11]:
v_2020_2 = data['Affected versions'].apply(pd.Series).eq('2020.2').any(1)
data_v2020_2_str = data[v_2020_2]["str_data"]

v_2020_3 = data['Affected versions'].apply(pd.Series).eq('2020.3').any(1)
data_v2020_3_str = data[v_2020_3]["str_data"]

/var/folders/l1/6p4_ds8168q6gcpff7fyvj_c0000gn/T/ipykernel_40167/3316689327.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  v_2020_2 = data['Affected versions'].apply(pd.Series).eq('2020.2').any(1)
/var/folders/l1/6p4_ds8168q6gcpff7fyvj_c0000gn/T/ipykernel_40167/3316689327.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  v_2020_3 = data['Affected versions'].apply(pd.Series).eq('2020.3').any(1)


In [12]:
t0 = time()

data["str_data"] = data["summary"].astype(str) + data["description"].astype(str)
data_docs_cleaned = data["str_data"].apply(cleaning_lambda)


time() - t0

29.442588806152344

In [13]:
data_samples = list(data_docs_cleaned)

### Векторизуем текст

In [14]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=N_FEATURES, stop_words="english"
)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 0.923s.



In [15]:
def grid_search_lda(search_params):
    lda = LatentDirichletAllocation(random_state=RANDOM_SEED)
    model = GridSearchCV(lda, param_grid=search_params, verbose=10)
    model.fit(tf)
    
    best_lda_model = model.best_estimator_

    print("Best Model's Params: ", model.best_params_)

    print("Best Log Likelihood Score: ", model.best_score_)

    print("Model Perplexity: ", best_lda_model.perplexity(tf))
    
    joblib.dump(best_lda_model, 'best_lda.pkl')
    return best_lda_model

In [16]:
def visualise_topics(data, model, tf_vectorizer, name):
    data_tf = tf_vectorizer.transform(data)
    pyLDAvis.enable_notebook()
    visualization = pyLDAvis.sklearn.prepare(model, data_tf, tf_vectorizer)
    pyLDAvis.save_html(visualization, name)
    return visualization

In [17]:
# search_params = {'n_components': [5, 6, 7, 8, 9, 10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
search_params = {'n_components': [5, 8, 10, 12, 15, 20]}
model = grid_search_lda(search_params)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START n_components=5..............................................
[CV 1/5; 1/6] END .........n_components=5;, score=-850971.094 total time= 1.0min
[CV 2/5; 1/6] START n_components=5..............................................
[CV 2/5; 1/6] END .........n_components=5;, score=-892233.097 total time= 1.1min
[CV 3/5; 1/6] START n_components=5..............................................
[CV 3/5; 1/6] END .........n_components=5;, score=-887515.008 total time= 1.1min
[CV 4/5; 1/6] START n_components=5..............................................
[CV 4/5; 1/6] END .........n_components=5;, score=-928375.728 total time= 1.0min
[CV 5/5; 1/6] START n_components=5..............................................
[CV 5/5; 1/6] END .........n_components=5;, score=-904276.174 total time=  59.7s
[CV 1/5; 2/6] START n_components=8..............................................
[CV 1/5; 2/6] END .........n_components=8;, score

In [18]:
visualise_topics(data_v2020_2_str, model, tf_vectorizer, "v2020_2.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.093027 -0.092063       1        1  11.668017
0      0.105765  0.083872       2        1  11.536576
5      0.077302  0.256683       3        1   8.547360
13     0.025111 -0.024175       4        1   8.528811
8      0.086045  0.036339       5        1   8.510701
9      0.056754 -0.054247       6        1   8.283914
10     0.146050  0.186567       7        1   7.839670
12    -0.166903 -0.016484       8        1   6.355073
3      0.060594  0.087281       9        1   5.967691
11     0.050956 -0.174956      10        1   5.459461
2      0.003846 -0.190848      11        1   4.042079
14    -0.234320  0.063684      12        1   3.928560
6      0.095158 -0.103204      13        1   3.731978
4      0.006388 -0.107247      14        1   3.072989
7     -0.405773  0.048796      15        1   2.527118, topic_info=       Term        Freq       Total Category  logprob  loglift
599      os  165.000000  165.000000  Default  30.0000  30.0000
848    step  245.000000  245.000000  Default  29.0000  29.0000
753  result  283.000000  283.000000  Default  28.0000  28.0000
273   error  189.000000  189.000000  Default  27.0000  27.0000
775  screen  128.000000  128.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
775  screen   53.755664  128.290885  Topic15  -2.3400   2.8082
458     jre   50.957026  140.506682  Topic15  -2.3934   2.6638
508    long    6.991193   25.972898  Topic15  -4.3798   2.3657
986    work    5.437031  220.370341  Topic15  -4.6312  -0.0240
365   handl    2.340925   11.135780  Topic15  -5.4739   2.1185

[754 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3         1  0.706157   absolut
4         2  1.049664  abstract
10        1  0.312923     activ
10        3  0.028448     activ
10        4  0.056895     activ
...     ...       ...       ...
992      10  0.049486       wsl
993      15  0.906923      xfff
994      15  1.133158   xffffff
996      15  1.007182        xx
997       4  0.658910      yaml

[2184 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 6, 14, 9, 10, 11, 13, 4, 12, 3, 15, 7, 5, 8])

In [19]:
visualise_topics(data_v2020_3_str, model, tf_vectorizer, "v2020_3.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.105765 -0.083872       1        1  14.768498
1     -0.093027  0.092063       2        1  11.861488
5     -0.077302 -0.256683       3        1   9.070867
13    -0.025111  0.024175       4        1   8.617395
8     -0.086045 -0.036339       5        1   8.447298
10    -0.146050 -0.186567       6        1   7.957502
3     -0.060594 -0.087281       7        1   6.851534
9     -0.056754  0.054247       8        1   6.149099
11    -0.050956  0.174956       9        1   5.347120
6     -0.095158  0.103204      10        1   4.979776
12     0.166903  0.016484      11        1   4.161806
14     0.234320 -0.063684      12        1   3.799206
2     -0.003846  0.190848      13        1   3.179202
7      0.405773 -0.048796      14        1   2.615045
4     -0.006388  0.107247      15        1   2.194164, topic_info=        Term        Freq       Total Category  logprob  loglift
599       os  130.000000  130.000000  Default  30.0000  30.0000
885     test  114.000000  114.000000  Default  29.0000  29.0000
848     step  201.000000  201.000000  Default  28.0000  28.0000
753   result  235.000000  235.000000  Default  27.0000  27.0000
273    error  135.000000  135.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
866  support    3.486730   43.531392  Topic15  -4.7090   1.2949
761      run    4.857119   95.955391  Topic15  -4.3775   0.8359
976     warn    3.978902  118.725934  Topic15  -4.5769   0.4236
591     open    2.996588  117.501763  Topic15  -4.8605   0.1504
595   option    2.483903   57.116818  Topic15  -5.0481   0.6841

[752 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         5  0.676493      abil
0        15  0.169123      abil
3         2  0.852846   absolut
4         1  1.038848  abstract
10        2  0.363449     activ
...     ...       ...       ...
992       6  0.183698       wsl
993      14  1.098304      xfff
994      14  0.915126   xffffff
996      14  0.997565        xx
997       4  0.796699      yaml

[2017 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 6, 14, 9, 11, 4, 10, 12, 7, 13, 15, 3, 8, 5])

## Вывод

Если не фильтровать текст кода, то токены из обрывков кода начинают заполнять все топики, поэтому приходится его полностью удалить.  

При обучении на количестве топиков от 5 до 20, лучший результат обучения по perplexity получился на 15 топиках.  
Однако, если посмотреть на визуализацию топиков для 2 разных релизов, то, во-первых, разделение на топики практически одинаковое, а, во-вторых, топики сложно интерпретировать и отделить друг от друга.  
Хотя, можно сделать вывод, что 12-я тема из второй визуализации ответственна за UI, тогда как темы в группе слева состоят из каких-то самых популярных слов и терминов для любого проекта.